# Linearna regresija

**"Linear approach to modelling the relationship between a scalar response and one or more explanatory variables"**

# Pregled
* Nadgledano učenje
* Prosta linearna regresija
* Polinomijalna regresija: polinom umesto linearne funkcije
* Višestruka linearna regresija: više feature-a
  * Hiperravni

#Pre svega, import

In [0]:
# TODO

# Nadgledano učenje

**Nadgledano učenje** (induktivno učenje, supervised learning) se bavi problemima u mašinskom učenju gde treba otkriti vezu između ulaznih i izlaznih podataka tj. "naučiti" funkciju koja mapira ulaze na izlaze, na osnovu datog skupa parova $(ulaz, izlaz)$. Tu funkciju nazivamo **hipoteza**. 

Dva glavna tipa problema su **regresija** (hipoteza je neprekidna funkcija) i **klasifikacija** (hipoteza je diskretna funkcija).

[Više o nadgledanom učenju](https://mcerovic.github.io/notes/SupervisedLearning/index.html)




# Linearna regresija

* **Linearna regresija** (Linear Regression) je jedna od najpoznatijih metoda nadgledanog učenja
* Za početak bavićemo se **prostom** linearnom regresijom  (simple LR, univariate LR)
  * Imamo jednu ulaznu promenljivu (**feature**) i jednu izlaznu promenljivu
* **Notacija**:
  * $x$: Ulazni podaci (features), $x_i$: Ulazni podatak iz i-tog trening uzorka (sempla)
  * $y$: Izlazni podaci (labels), $y_i$: Izlazni podatak iz i-tog trening uzorka (sempla)
  * $m$: Veličina trening skupa - broj semplova
* **Hipoteza (model)**: $ h_\theta(x) = \theta_0 + \theta_1 x $
  * Linearna funkcija!
  * "Učimo" parametre $\theta_0$ i $\theta_1$, tj. modifikujemo ih tako da budu "što bolji"
  * Kako znamo kada su "bolji"?
* **Funkcija troška** (kriterijumska funkcija, funkcija koštanja, cost function, loss function): **MSE** (srednje kvadratno odstupanje, mean squared error)
  * $J(\theta_0, \theta_1) = \frac{1}{m} \sum\limits_{i=1}^{m} (h_\theta(x_i) - y_i)^2$ (nekada $\frac{1}{2m}$ zbog lepšeg izvoda)
 * Dakle, menjamo $\theta_0$ i $\theta_1$ tako da **minimizujemo** funkciju troška, kako?
* **Optimizacija: gradijentni spust** (gradient descent): iterativni optimizacioni metod za minimizaciju funkcije
    * Određuje tačno u kom smeru treba da se krećemo da što efikasnije poboljšamo vrednosti parametara
    * Varijante: Batch, Minibatch, Stochastic
* [Više o linearnoj regresiji](https://mcerovic.github.io/notes/LinearRegression/index.html)
* [Više o gradijentnom spustu](https://mcerovic.github.io/notes/GradientDescent/index.html)

# Primer: predviđanje očekivanog životnog veka (OŽV) na osnovu nataliteta

Istraživanje od strane World Bank:  [World Development Indicators](https://datacatalog.worldbank.org/dataset/world-development-indicators).

[Prateća vizualizacija](https://www.google.com/publicdata/explore?ds=d5bncppjof8f9_&ctype=b&strail=false&nselm=s&met_x=sp_dyn_le00_in&scale_x=lin&ind_x=false&met_y=sp_dyn_tfrt_in&scale_y=lin&ind_y=false&met_s=sp_pop_totl&scale_s=lin&ind_s=false&dimp_c=country:region&ifdim=country&iconSize=0.5&uniSize=0.035#!ctype=b&strail=false&bcs=d&nselm=s&met_x=sp_dyn_le00_in&scale_x=lin&ind_x=false&met_y=sp_dyn_tfrt_in&scale_y=lin&ind_y=false&met_s=sp_pop_totl&scale_s=lin&ind_s=false&dimp_c=country:region&ifdim=country&pit=1421395200000&hl=en_US&dl=en_US&ind=false)

Ovo ukazuje na određenu korelaciju (veći natalitet => manji životni vek). Pokušaćemo da kvantifikujemo ovaj odnos tj. da naučimo funkciju $OŽV = f(natalitet)$.


In [0]:
# TODO

# Višestruka linearna regresija

* **Višestruka linearna regresija** (Multiple Linear Regression, nekad i Multivariate Linear Regression) je ime za linearnu regresiju gde radimo sa više od jednim ulaznim feature-om.
* Kao primer radićemo sa datasetom sa sajta **kaggle** pod imenom [**80 cereals**](https://www.kaggle.com/crawford/80-cereals).
* Dati su razni podaci o 80 tipova cerealija sa kojima možemo raditi svašta... Recimo da želimo da predvidimo promenljivu **calories** na osnovu promenljivih **fat** i **sugars**.


---



* Dodatna notacija: $x^{(i)}$ je i-ti feature
* Hipoteza je sada: $ h_\theta(x) = bias + w_0 x^{(0)} + w_1 x^{(1)} + ... + w_{n-1} x^{(n-1)} $
  * I dalje linearna funkcija samo po više promenlijvih! Umesto fitovanja prave fitujemo ravan tj. hiperravan u određenom broju dimenzija.
  * "Učimo" parametre $w$!
* Funkcija troška ostaje ista, kao i metod optimizacije.


---


* Kod ćemo pisati generički tako da radi sa bilo kojim brojem feature-a.
* Poći ćemo od prethodnog koda.


In [0]:
# TODO

# Polinomijalna regresija

* **Polinomijalna regresija** (Polynomial Regression) je naziv za regresiju u kojoj umesto linearne funkcije, učimo polinom nekog stepena.
* Izraženo preko višestruke linearne regresije: od stepenova feature-a pravimo nove feature i onda radimo istu proceduru kao u višestrukoj regresiji.
* Cela postavka dakle ostaje ista kao u višestrukoj linearnoj regresiji, samo je priprema feature-a drukčija.


---

* Izmenimo primer sa natalitetom i OŽV da podržimo polinome proizvoljnog stepena i posmatrajmo kako to utiče na rezultat.

In [0]:
# TODO